In [101]:
import neurokit2 as nk
import numpy as np
from scipy.signal import find_peaks
import json

In [102]:
def pair_peaks(peak_array_1, peak_array_2, max_dist=50):
    """
    Pair peaks from two arrays by matching closest indices within max_dist (samples).
    Returns matched peaks (trimmed arrays).
    """
    matched_1 = []
    matched_2 = []
    i, j = 0, 0
    while i < len(peak_array_1) and j < len(peak_array_2):
        diff = peak_array_2[j] - peak_array_1[i]
        if abs(diff) <= max_dist:
            matched_1.append(peak_array_1[i])
            matched_2.append(peak_array_2[j])
            i += 1
            j += 1
        elif peak_array_1[i] < peak_array_2[j]:
            i += 1
        else:
            j += 1
    return np.array(matched_1), np.array(matched_2)


In [103]:
def extract_features(ecg_signal, fs, subject_type="fetal"):
    signals, info = nk.ecg_process(ecg_signal, sampling_rate=fs)
    features = nk.ecg_analyze(signals, sampling_rate=fs)

    summary = {
        "ECG_Rate_Mean":  float(features.get("ECG_Rate_Mean", 0)),
        "HRV_SDNN":       float(features.get("HRV_SDNN", 0)),
        "HRV_RMSSD":      float(features.get("HRV_RMSSD", 0)),
        "HRV_pNN50":      float(features.get("HRV_pNN50", 0)),
    }

    q = signals.index[signals["ECG_Q_Peaks"] == 1].to_numpy()
    s = signals.index[signals["ECG_S_Peaks"] == 1].to_numpy()
    p = signals.index[signals["ECG_P_Peaks"] == 1].to_numpy()
    t = signals.index[signals["ECG_T_Offsets"] == 1].to_numpy()
    r = signals.index[signals["ECG_R_Peaks"] == 1].to_numpy()

    flags = []

    # QRS Duration
    if len(q) > 0 and len(s) > 0:
        q_matched, s_matched = pair_peaks(q, s)
        if len(q_matched) > 0:
            qrs = ((s_matched - q_matched) / fs * 1000)
            summary["ECG_QRS_Duration"] = float(np.mean(qrs))
        else:
            summary["ECG_QRS_Duration"] = np.nan
            flags.append("Could not calculate QRS Duration - insufficient peak matching")
    else:
        summary["ECG_QRS_Duration"] = np.nan
        flags.append("Could not calculate QRS Duration - missing Q or S peaks")

    # QT Interval
    if len(q) > 0 and len(t) > 0:
        q_matched, t_matched = pair_peaks(q, t)
        if len(q_matched) > 0:
            qt = ((t_matched - q_matched) / fs * 1000)
            summary["ECG_QT_Interval"] = float(np.mean(qt))
        else:
            summary["ECG_QT_Interval"] = np.nan
            flags.append("Could not calculate QT Interval - insufficient peak matching")
    else:
        summary["ECG_QT_Interval"] = np.nan
        flags.append("Could not calculate QT Interval - missing Q or T peaks")

    # PR Interval
    if len(p) > 0 and len(q) > 0:
        p_matched, q_matched = pair_peaks(p, q)
        if len(p_matched) > 0:
            pr = ((q_matched - p_matched) / fs * 1000)
            summary["ECG_PR_Interval"] = float(np.mean(pr))
        else:
            summary["ECG_PR_Interval"] = np.nan
            flags.append("Could not calculate PR Interval - insufficient peak matching")
    else:
        summary["ECG_PR_Interval"] = np.nan
        flags.append("Could not calculate PR Interval - missing P or Q peaks")

    # Your existing abnormality flags (e.g. HR ranges, QRS, QT limits)
    if subject_type == "fetal":
        if summary["ECG_Rate_Mean"] < 110:
            flags.append("Fetal bradycardia (HR < 110 bpm)")
        elif summary["ECG_Rate_Mean"] > 160:
            flags.append("Fetal tachycardia (HR > 160 bpm)")
        if not np.isnan(summary["ECG_QRS_Duration"]) and summary["ECG_QRS_Duration"] > 120:
            flags.append("Prolonged QRS duration (Possible conduction issue)")

    elif subject_type == "maternal":
        if summary["ECG_Rate_Mean"] < 60:
            flags.append("Maternal bradycardia (HR < 60 bpm)")
        elif summary["ECG_Rate_Mean"] > 100:
            flags.append("Maternal tachycardia (HR > 100 bpm)")
        if not np.isnan(summary["ECG_QRS_Duration"]) and summary["ECG_QRS_Duration"] > 120:
            flags.append("Wide QRS complex (> 120 ms)")
        if not np.isnan(summary["ECG_QT_Interval"]) and summary["ECG_QT_Interval"] > 450:
            flags.append("Prolonged QT interval (Risk of arrhythmia)")
        if not np.isnan(summary["ECG_PR_Interval"]) and summary["ECG_PR_Interval"] > 200:
            flags.append("Prolonged PR interval (Possible AV block)")

    summary["Flags"] = flags
    return summary

In [104]:
base_dir = "ica_cleaned_signals"
output_dir = "metrics_json"
os.makedirs(output_dir, exist_ok=True)

# Example: list all npy files in your directory
files = [f for f in os.listdir(base_dir) if f.endswith(".npy")]

fs = 250  # sampling frequency

for filename in files:
    filepath = os.path.join(base_dir, filename)

    # Determine fetal/maternal from filename (assumes filename contains 'fecg' or 'mecg')
    if "fecg" in filename:
        subject_type = "fetal"
    elif "mecg" in filename:
        subject_type = "maternal"
    else:
        print(f"Skipping unknown type for file {filename}")
        continue

    # Load signal
    signal = np.load(filepath)

    # Extract features
    try:
        features = extract_features(signal, fs, subject_type)

        # Add metadata for clarity
        features["subject_type"] = subject_type
        features["filename"] = filename

        # Save to JSON
        json_filename = filename.replace(".npy", "_features.json")
        json_path = os.path.join(output_dir, json_filename)

        with open(json_path, "w") as json_file:
            json.dump(features, json_file, indent=4)

        print(f"Saved features for {filename} to {json_filename}")

    except Exception as e:
        print(f"Error processing {filename}: {e}")

/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "ECG_Rate_Mean":  float(features.get("ECG_Rate_Mean", 0)),
/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "HRV_SDNN":       float(features.get("HRV_SDNN", 0)),
/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "HRV_RMSSD":      float(features.get("HRV_RMSSD", 0)),
/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:9: FutureWarning: Calling float on a single element Series is deprecated

Saved features for sub07_l1_c0_combined_cleaned_fecg.npy to sub07_l1_c0_combined_cleaned_fecg_features.json


/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "ECG_Rate_Mean":  float(features.get("ECG_Rate_Mean", 0)),
/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "HRV_SDNN":       float(features.get("HRV_SDNN", 0)),
/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "HRV_RMSSD":      float(features.get("HRV_RMSSD", 0)),
/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:9: FutureWarning: Calling float on a single element Series is deprecated

Saved features for sub06_l3_c0_combined_cleaned_fecg.npy to sub06_l3_c0_combined_cleaned_fecg_features.json


/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "ECG_Rate_Mean":  float(features.get("ECG_Rate_Mean", 0)),
/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "HRV_SDNN":       float(features.get("HRV_SDNN", 0)),
/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "HRV_RMSSD":      float(features.get("HRV_RMSSD", 0)),
/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:9: FutureWarning: Calling float on a single element Series is deprecated

...

/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "ECG_Rate_Mean":  float(features.get("ECG_Rate_Mean", 0)),
/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "HRV_SDNN":       float(features.get("HRV_SDNN", 0)),
/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "HRV_RMSSD":      float(features.get("HRV_RMSSD", 0)),
/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:9: FutureWarning: Calling float on a single element Series is deprecated

Saved features for sub03_l2_c0_combined_cleaned_fecg.npy to sub03_l2_c0_combined_cleaned_fecg_features.json


/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "ECG_Rate_Mean":  float(features.get("ECG_Rate_Mean", 0)),
/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "HRV_SDNN":       float(features.get("HRV_SDNN", 0)),
/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "HRV_RMSSD":      float(features.get("HRV_RMSSD", 0)),
/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:9: FutureWarning: Calling float on a single element Series is deprecated

Saved features for sub08_l5_c0_combined_cleaned_mecg.npy to sub08_l5_c0_combined_cleaned_mecg_features.json
Saved features for sub04_l4_c0_combined_cleaned_mecg.npy to sub04_l4_c0_combined_cleaned_mecg_features.json


/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "ECG_Rate_Mean":  float(features.get("ECG_Rate_Mean", 0)),
/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "HRV_SDNN":       float(features.get("HRV_SDNN", 0)),
/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "HRV_RMSSD":      float(features.get("HRV_RMSSD", 0)),
/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/3597693706.py:9: FutureWarning: Calling float on a single element Series is deprecated